In [17]:
#spark.stop()

In [18]:
import os, sys
import datetime as dt
DEVEL_SRC = os.path.join(os.environ.get('BDA_USER_HOME', ''), "src", "devel")
if DEVEL_SRC not in sys.path:
    sys.path.append(DEVEL_SRC)

USECASES_SRC = os.path.join(DEVEL_SRC, "use-cases") # TODO when - is removed, remove also this line and adapt imports
if USECASES_SRC not in sys.path: 
    sys.path.append(USECASES_SRC)
    
AMDOCS_SRC = os.path.join(DEVEL_SRC, "amdocs_informational_dataset") # TODO when - is removed, remove also this line and adapt imports
if AMDOCS_SRC not in sys.path: 
    sys.path.append(AMDOCS_SRC)
    
import pykhaos.utils.custom_logger as clogger
logging_file = os.path.join(os.environ.get('BDA_USER_HOME', ''), "logging",
                                    "out_" + dt.datetime.now().strftime("%Y%m%d_%H%M%S") + ".log")
logger = clogger.configure_logger(log_filename=logging_file, std_channel=sys.stderr, logger_name="")
logger.info("Logging to file {}".format(logging_file))    
    
    
from project.project_generic import Project


import pykhaos.utils.notebooks as nb

project_obj = Project("CCC model", "CCC model")   

RUNNING_FROM_NOTEBOOK = nb.isnotebook()
import matplotlib.pyplot as plt
if RUNNING_FROM_NOTEBOOK:
    %load_ext autoreload
    %autoreload 2
    %matplotlib inline  
    
#logger = my_project.logger

if not RUNNING_FROM_NOTEBOOK:
    args = my_project.arg_parser()

20181119-082337 [INFO ] Logging to file /var/SP/data/home/csanc109/logging/out_20181119_082337.log


_initialize spark
Ended spark session: 31.3939368725 secs | default parallelism=2
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
spark=project_obj.spark

In [20]:
%load_ext autoreload
%autoreload 2
%matplotlib inline 
from churn.datapreparation.data_loader import get_port_requests_table
from churn.datapreparation.config import Config

config_obj = Config(None)

port_start_date = "20180901"
port_end_date = "20180930"
ref_date = "20181021"

select_cols = ["portout_date",  "msisdn_a", "label", "days_from_portout"]

#df_port = get_port_requests_table(spark, config_obj, port_start_date, port_end_date, ref_date, select_cols)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
/var/SP/data/home/csanc109/src/devel/use-cases/churn
Reading config from file /var/SP/data/home/csanc109/src/devel/use-cases/churn/input/datapreparation_churn.yaml
Reading internal config from file /var/SP/data/home/csanc109/src/devel/use-cases/churn/datapreparation/config/internal_config.yaml
----- CHECKING INPUT PARAMETERS ------
if
elif
20180807
if
20180814
if
20180821
else
20180831
if
elif
20180907
if
20180914
if
20180921
else
20180930
('20180731', 8, '20180930')
----- INPUT PARAMETERS OK! ------
{'closing_day': 20180731,
 'cycles_horizon': 8,
 'discard_vars': ['order_id'],
 'discarded_cycles': 4,
 'internal_config_file': '/var/SP/data/home/csanc109/src/devel/use-cases/churn/datapreparation/config/internal_config.yaml',
 'labeled': True,
 'level': 'service',
 'model_target': 'port',
 'save_car': True,
 'segment_filter': 'mobileandfbb',
 'service_set': 'mobile',
 'sources': {'ids': {'address': Fa

In [21]:
portout_dates_list = df_port.select('portout_date').distinct().collect()
portout_dates_min = min(portout_dates_list)
portout_dates_max = max(portout_dates_list)

ccc_start_date = "20180801"
ccc_end_date = "20180930"

NameError: name 'df_port' is not defined

In [ ]:
from engine.call_centre_calls import CallCentreCalls

#Call Centre Calls
ccc = CallCentreCalls(spark)
df_ccc = ccc.get_ccc_service_df(ccc_end_date,ccc_start_date)
df_all = ccc.all_interactions


In [13]:
# HDFS_DIR_DATA = '/user/csanc109/projects/churn/data/ccc_model/'
# df_all = df_all.repartition(1)
# df_all.write.mode('overwrite').format('csv').option("header", "true").save('/user/csanc109/projects/churn/data/ccc_model/all_interactions_20181114.csv')

In [10]:
# LAS QUE NO CRUZAN
# from pyspark.sql.functions import desc, upper
# df_nocruzan=df_all.where(col("Bucket")=="NA").groupby("source", "INT_Tipo", "INT_Subtipo", "INT_Razon", "INT_Resultado", "X_WORKGROUP", "direction", "type_td").agg(sql_count("*").alias("count")).sort(desc("count"))
# df_nocruzan.show(100)

In [ ]:
#### EJECUTAR SOLO EN CASO DE NECESIDAD!!!!!!
# df_pandas=df_nocruzan.limit(100).toPandas()
# writer = pd.ExcelWriter("/var/SP/data/home/csanc109/data/churn/ccc/nocruzan_20181114.xlsx", engine='xlsxwriter')
# df_pandas.to_excel(writer, startrow = 0, sheet_name='Bucket NA')
# writer.save()

In [440]:
# Para ver los buckets a NA NA|2269376
df_all.select('Bucket').groupby('Bucket').count().sort('Bucket').show()

+--------------------+-------+
|              Bucket|  count|
+--------------------+-------+
|             Bucket_|     76|
|Bucket_Billing - ...| 887119|
|Bucket_Churn/Canc...|1409282|
|  Bucket_Collections| 273738|
|Bucket_DSL/FIBER ...| 781377|
|Bucket_Device del...| 179853|
|Bucket_Device upg...| 259134|
|Bucket_New adds p...| 707436|
|Bucket_Other cust...| 630229|
|Bucket_Prepaid ba...|  10286|
|Bucket_Product an...| 927302|
|Bucket_Quick Closing|  76563|
|Bucket_Tariff man...| 169176|
|Bucket_Voice and ...| 251504|
|                  NA|2210417|
+--------------------+-------+



In [442]:
from pyspark.sql.functions import sum as sql_sum
# df_all distinct count 2719030 df_sinNA distinct count 1836042
df_sinNA = ((df_all
            .groupby('msisdn')
            .agg(sql_sum(when(col('Bucket')=="NA", lit(1)).otherwise(lit(0))).alias("bucket_sum")))
            .where(col("bucket_sum")==0))
df_sinNA.count()

1847227

In [36]:
from pyspark.sql.functions import collect_set, concat, size, coalesce, col, lpad, struct, count as sql_count, lit, min as sql_min, max as sql_max, collect_list, udf, when
from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField, IntegerType

from engine.general_functions import format_date, compute_diff_days


df_agg = (df_all.where(col("Bucket")!="NA")\
            .withColumn("fx_interaction",concat(col('year'),lpad(col('month'),2, '0'),lpad(col('day'),2, '0')))
            .withColumn("tuples", struct(["fx_interaction","Bucket"]))
            .groupby('msisdn')
            .agg(sql_count(lit(1)).alias("num_interactions"),
                 sql_min(col('fx_interaction')).alias("first_interaction"),
                 sql_max(col('fx_interaction')).alias("latest_interaction"),
                 sql_max(col("tuples")).alias("tuples_max"), # latest interaction: [max(fx_interaction), bucket]
                 sql_min(col("tuples")).alias("tuples_min"), # first interaction: [min(fx_interaction), bucket]
                 collect_list('Bucket').alias("bucket_list"),
                 collect_set('Bucket').alias("bucket_set")
             ))

df_agg = df_agg.withColumn("ref_date", format_date(lit(ref_date)))
df_agg = (df_agg.withColumn("bucket_1st_interaction", col("tuples_min")["Bucket"])
                .withColumn("bucket_latest_interaction", col("tuples_max")["Bucket"])
                .withColumn("nb_diff_buckets", size("bucket_set"))
                .withColumn("num_interactions", size("bucket_list"))
          .drop(*['tuples_max','tuples_min'])
         )

for cc in ["first_interaction", "latest_interaction"]:
    df_agg = (df_agg.withColumn("fx_{}".format(cc), format_date(cc, filter_dates_1900=True))  # days before 1900 converted to None
                    .withColumn("days_since_{}".format(cc), compute_diff_days("fx_{}".format(cc), "ref_date")))
    
from collections import Counter
from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField

def get_mode(lst):
    dd = Counter(lst).most_common(2)
    return dd[0][0] if (len(dd)==1 or dd[0][1] > dd[1][1]) else "TIE"
def get_mode_freq(lst):
    dd = Counter(lst).most_common(2)
    return dd[0][1] # value

get_mode_udf = udf(lambda lst: get_mode(lst), StringType())
get_mode_freq_udf = udf(lambda lst: get_mode_freq(lst), IntegerType())

df_agg = (df_agg.withColumn("most_common_bucket_with_ties", when(coalesce(size(col("bucket_list")),lit(0))==0,"None").otherwise(get_mode_udf(col("bucket_list"))))
                .withColumn("most_common_bucket", when(col("most_common_bucket_with_ties").rlike("^TIE"), col('bucket_latest_interaction')).otherwise(col("most_common_bucket_with_ties")))
                .withColumn("most_common_bucket_interactions", when(coalesce(size(col("bucket_list")),lit(0))==0,-1).otherwise(get_mode_freq_udf(col("bucket_list"))))
          )



In [37]:
df_agg.where(col("msisdn")==605256544).select("bucket_list","bucket_latest_interaction","most_common_bucket_with_ties", "most_common_bucket").show(10, False)

+------------------------------------------------------------------------------------------------------------------------+-------------------------+----------------------------+-------------------------+
|bucket_list                                                                                                             |bucket_latest_interaction|most_common_bucket_with_ties|most_common_bucket       |
+------------------------------------------------------------------------------------------------------------------------+-------------------------+----------------------------+-------------------------+
|[Bucket_Product and Service management, Bucket_Billing - Postpaid, Bucket_Tariff management, Bucket_Churn/Cancellations]|Bucket_Billing - Postpaid|TIE                         |Bucket_Billing - Postpaid|
+------------------------------------------------------------------------------------------------------------------------+-------------------------+----------------------------+-------

In [245]:
tablita = spark.read.load('/user/csanc109/projects/churn/port/mobileandfbb/df_port_mobileandfbb_service_20181021_c8_d4')

In [123]:
from churn.utils.constants import *
import os
import sys
from pykhaos.utils.date_functions import move_date_n_cycles
import datetime as dt       
    
closing_day = "20180731"
cycles_horizon = 8
end_date = move_date_n_cycles(closing_day, n=cycles_horizon)
print(closing_day, cycles_horizon, end_date)

if end_date > dt.date.today().strftime("%Y%m%d"):
    print("Program cannot be run with cycles={0} since closing_day + {0} cycles is in the future {1}".format(cycles_horizon, end_date))
    sys.exit(1)

('20180731', 8, '20180930')


In [124]:
from churn.datapreparation.config import Config
config_obj = Config(filename='/var/SP/data/home/csanc109/src/devel/use-cases/churn/models/ccc/input/ccc_train_test.yaml',
                   check_args=False)

from churn.datapreparation.data_loader import get_port_requests_table
select_cols = ["msisdn_a", "portout_date"]
df_port = get_port_requests_table(spark, config_obj=config_obj, start_date="20180901", end_date="20180930",
                                  ref_date="20181021", select_cols=select_cols)

print("Number of portouts in range {}".format(df_port.count()))
dist_portouts_dates = df_port.select("portout_date").distinct().rdd.map(lambda x: x["portout_date"]).collect()

from amdocs_informational_dataset.engine.call_centre_calls import CallCentreCalls
ccc = CallCentreCalls(spark)

Reading config from file /var/SP/data/home/csanc109/src/devel/use-cases/churn/models/ccc/input/ccc_train_test.yaml
Reading internal config from file /var/SP/data/home/csanc109/src/devel/use-cases/churn/datapreparation/config/internal_config.yaml
{'campaign_creation': {'campaign_date': '201810',
                       'create_csv_file': True,
                       'storage_format': 'hdfs'},
 'data_preparation': {'ccc_period': 60,
                      'force_generation': False,
                      'level': 'service',
                      'model_target': 'port',
                      'port_period': [20180901, 20180930],
                      'save': 'true,',
                      'segment_filter': 'mobileandfbb',
                      'service_set': 'mobile'},
 'internal_config_file': '/var/SP/data/home/csanc109/src/devel/use-cases/churn/datapreparation/config/internal_config.yaml',
 'predict': {'dd_prepago': [201808],
             'do_predict': 'true,',
             'use_train_model

# CCC model

In [ ]:
def union_all(dfs):
    if len(dfs) > 1:
        return dfs[0].unionAll(union_all(dfs[1:]))
    else:
        return dfs[0]

from pykhaos.utils.date_functions import move_date_n_days
n_ccc = -60 # FIXME -abs(config_obj.['data_preparation'][60])
dfs = []
from pyspark.sql.functions import col
from churn.models.ccc.data.ccc_preparation import fill_missing_buckets
counter = 0

for port_date in dist_portouts_dates:
    
    print(port_date)
    msisdn_list = df_port.where(col("portout_date")==port_date).select("msisdn_a").distinct().rdd.map(lambda x: x["msisdn_a"]).collect()
    yyyymmdd_portout_date = str(port_date.split(" ")[0].replace("-",""))
    
    print("There are {} services with portout_date {}".format(len(msisdn_list), yyyymmdd_portout_date))
    
    ccc_start_date = move_date_n_days(yyyymmdd_portout_date, n=n_ccc, str_fmt="%Y%m%d")
    ccc_end_date = yyyymmdd_portout_date
    _ = ccc.get_ccc_service_df(ccc_end_date, ccc_start_date).where(col("msisdn").isin(msisdn_list))
    
    df_all = fill_missing_buckets(spark, ccc.all_interactions)
    ccc.all_interactions = df_all
    df_agg = ccc.add_l2_ccc_variables(self, closing_day)
    
    dfs.append(df_agg)
    counter = counter+1
    
    if counter==2: break

df_agg_all = union_all(dfs)

print("Number of rows {}".format(df_agg_all.count()))

df_all_model = df_all.select("msisdn", "most_common_bucket")

# Add data to list

In [24]:
from amdocs_informational_dataset.engine.call_centre_calls import CallCentreCalls
from pyspark.sql.functions import collect_set, concat, size, coalesce, col, lpad, struct, count as sql_count, lit, min as sql_min, max as sql_max, collect_list, udf, when
from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField, IntegerType


In [134]:
from amdocs_informational_dataset.engine.call_centre_calls import CallCentreCalls
from pyspark.sql.functions import collect_set, concat, size, coalesce, col, lpad, struct, count as sql_count, lit, min as sql_min, max as sql_max, collect_list, udf, when
from pyspark.sql.types import StringType, ArrayType, MapType, StructType, StructField, IntegerType

#ccc = CallCentreCalls(spark)

ccc_end_date_ = "20181031"
ccc_start_date_ = "20181001"

closing_day = "20181031"
df_agg = CallCentreCalls(spark).get_ccc_service_df(ccc_end_date_, ccc_start_date_)


Getting Call Centre Calls Information...
Calling to add_l2_ccc_variables....


In [135]:
from pyspark.sql.functions import array, regexp_extract

df_agg = df_agg.withColumn("IND_PBMA_SRV", when(col("num_averias")+col("num_incidencias")>0, 1).otherwise(0))

cols_averia = [col_ for col_ in df_agg.columns if col_.startswith("ccc_Raw_Averia")]
cols_incidencia = [col_ for col_ in df_agg.columns if col_.startswith("ccc_Raw_Incidencia")]


COLS_PROB = cols_averia + cols_incidencia
# Return the column name of the column with max value
colname_with_max_udf = udf(lambda milist: COLS_PROB[
        sorted([(vv, idx) for idx, vv in enumerate(milist)], key=lambda tup: tup[0], reverse=True)[0][1]],
                               StringType())

df_agg = df_agg.withColumn("DETALLE_PBMA_SRV", when(col("IND_PBMA_SRV")>0, colname_with_max_udf(array(COLS_PROB))).otherwise("None"))
df_agg = df_agg.withColumn("DETALLE_PBMA_SRV", when(col("DETALLE_PBMA_SRV").like("ccc_Raw_%"), regexp_extract(col('DETALLE_PBMA_SRV'), "^ccc_Raw_(.*)$", 1)).otherwise(col('DETALLE_PBMA_SRV')))

In [136]:
df_agg.where(col("IND_PBMA_SRV")>0).select(["DETALLE_PBMA_SRV"]+COLS_PROB).show(10)

+--------------------+------------------+--------------------+-----------------+--------------------+-------------------+---------------------------+---------------------------------+----------------------------------+--------------------------------+--------------------------+----------------------+------------------------+----------------------------------+
|    DETALLE_PBMA_SRV|ccc_Raw_Averia_DSL|ccc_Raw_Averia_Fibra|ccc_Raw_Averia_TV|ccc_Raw_Averia_Resto|ccc_Raw_Averia_Neba|ccc_Raw_Averia_Modem_Router|ccc_Raw_Incidencia_Provision_Neba|ccc_Raw_Incidencia_Provision_Fibra|ccc_Raw_Incidencia_Provision_DSL|ccc_Raw_Incidencia_Tecnica|ccc_Raw_Incidencia_SGI|ccc_Raw_Incidencia_Resto|ccc_Raw_Incidencia_Provision_Movil|
+--------------------+------------------+--------------------+-----------------+--------------------+-------------------+---------------------------+---------------------------------+----------------------------------+--------------------------------+-------------------------

In [137]:
fichero="/user/csanc109/projects/churn/preds_comb_20181031_all.txt" # msisdn es msisdn_d
df_scores = spark.read.option("delimiter", "|").option("header", True).csv(fichero)
df_scores = df_scores.withColumnRenamed("msisdn", "msisdn_d_scores")

hdfs_write_path_common = '/data/udf/vf_es/amdocs_inf_dataset/'
hdfs_partition_path = 'year=' + str(int(ccc_end_date_[:4])) + '/month=' + str(int(ccc_end_date_[4:6])) + '/day=' + str(int(ccc_end_date_[6:8]))
path_service = hdfs_write_path_common +'service/'+hdfs_partition_path
df_service = (spark.read.load(path_service)).select("msisdn", "campo2").withColumnRenamed("msisdn", "msisdn_a_service")

df_scores_join = df_scores.join(df_service, on=df_scores["msisdn_d_scores"]==df_service["campo2"], how="left")

df_scores_incidencias = df_scores_join.join(df_agg, on=df_scores_join["msisdn_a_service"]==df_agg["msisdn"], how="left")

# pyspark cannot write arrays to csv's
from pyspark.sql.functions import concat_ws
df_scores_incidencias = df_scores_incidencias.withColumn('bucket_list', concat_ws(",", col('bucket_list'))).withColumn('bucket_set', concat_ws(",", col('bucket_set')))

df_scores_incidencias = df_scores_incidencias.fillna( { 'IND_PBMA_SRV':0, 'DETALLE_PBMA_SRV':"None" } )
for col_ in cols_averia+cols_incidencia+["num_incidencias", "num_averias", "num_interactions", 
                                         "days_since_first_interaction", "days_since_latest_interaction", 
                                         "num_ivr_interactions"]:
    df_scores_incidencias = df_scores_incidencias.fillna( { col_:0} )
    
cols_problemas = [] # Rename columns to make names more readable
for col_ in cols_averia+cols_incidencia:
    col_renamed=re.match("^ccc_Raw_(.*)$", col_).group(1)
    df_scores_incidencias = df_scores_incidencias.withColumnRenamed(col_, col_renamed)
    cols_problemas.append(col_renamed)

In [138]:
df_scores_incidencias = df_scores_incidencias.drop("msisdn") # anonimized
df_scores_incidencias = df_scores_incidencias.withColumnRenamed("msisdn_d_scores", "msisdn") # deanonimized

In [139]:
df_scores_incidencias = df_scores_incidencias.withColumn("comb_score", col("comb_score").cast(FloatType()))
df_scores_incidencias = df_scores_incidencias.orderBy('comb_score', ascending=False)

df_scores_incidencias = df_scores_incidencias.repartition(1)
# df_scores_incidencias.select(*['msisdn', 'comb_score', 'comb_decile', 'IND_PBMA_SRV', 'DETALLE_PBMA_SRV', 'num_averias', 'num_incidencias', 'num_interactions', 'days_since_first_interaction',
#  'days_since_latest_interaction', 'num_ivr_interactions'] + cols_problemas).write.mode('overwrite').format('csv').option('sep', '|').option('header', 'true').save('/tmp/csanc109/churn/preds_comb_20181031_all_incidences.csv.v13')

df_scores_incidencias.select(*['msisdn', 'comb_score', 'comb_decile', 'IND_PBMA_SRV', 'DETALLE_PBMA_SRV']).write.mode('overwrite').format('csv').option('sep', '|').option('header', 'true').save('/tmp/csanc109/churn/preds_comb_20181031_all_incidences.csv.v14')

In [140]:
df_scores_incidencias.select("comb_score").show(10)

+----------+
|comb_score|
+----------+
|  0.995526|
|  0.995526|
|  0.995526|
|  0.995526|
|  0.995525|
|  0.995525|
|  0.995525|
|  0.995525|
|  0.995524|
|  0.995524|
+----------+
only showing top 10 rows



In [109]:
# def create_hist(rdd_histogram_data):
#     heights = np.array(rdd_histogram_data[1])
#     full_bins = rdd_histogram_data[0]
#     mid_point_bins = full_bins[:-1]
#     widths = [abs(i - j) for i, j in zip(full_bins[:-1], full_bins[1:])]
#     bar = plt.bar(mid_point_bins, heights, width=widths, color='b')
#     return bar

# rdd_histogram_data = df_scores_incidencias.select("score").rdd.flatMap(lambda x: x).histogram(20)
# rdd_histogram_data.show()

AttributeError: 'tuple' object has no attribute 'show'